In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import utils
import rcnn
from torch.utils.data import DataLoader,Dataset
from PIL import Image,ImageDraw
import os
import json
from torchvision import transforms
import skimage
from train import train_step
import torch.optim as optim

epochs=4
batch_size=16
device='cuda'

In [2]:
model=rcnn.FasterRCNN(device=device,mode='train')
model.to(device)

transform=transforms.Compose([
    transforms.ToTensor()
])
trainset=utils.MyDataset('./train',transform=transform)

trainloader = DataLoader(dataset=trainset,
                         batch_size=batch_size,
                         shuffle=True,
                         collate_fn=utils.collate_fn,
                         drop_last=True)


C:\Users\Napoleon\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Napoleon\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)
#optimizer = optim.SGD(model.parameters(), lr=0.001)

In [4]:
model.mode='train'
model.train()
# 总共解决了几个问题：
# 1、 训练结果预测输出全是背景；解决：需要对rpn网络进行正负样本平衡
# 2、 训练过程中，box的损失波动较大；解决：去掉一些出界，过小的box，让用于回归训练的输出更加优质
# 
for param in model.backbone.parameters():
    param.requires_grad = False

for i in range(epochs):
    print('epoch ',i)
    batch_loss,batch_logit_loss,batch_box_loss=train_step(trainloader,model,optimizer,mode='total')
    print('batch_loss:',batch_loss.item())
    print('batch_logit_loss:',batch_logit_loss.item())
    print('batch_box_loss:',batch_box_loss.item())
    # box_loss.backward(retain_graph=True)
    # optimizer.step()
    # print('box loss:', box_loss.item())
    #
    # logit_loss.backward()
    # optimizer.step()
    # print('logit loss:', logit_loss.item())
print('finish!')

epoch  0
item loss: 50.64851760864258
box loss: 50.60029220581055
logit loss: 0.04822378233075142
item loss: 0.029400024563074112
box loss: 0.0
logit loss: 0.029400024563074112
item loss: 0.03445080295205116
box loss: 0.0
logit loss: 0.03445080295205116
item loss: 0.027668830007314682
box loss: 0.0
logit loss: 0.027668830007314682
item loss: 41.61325454711914
box loss: 41.573211669921875
logit loss: 0.04004350304603577
item loss: 50.426109313964844
box loss: 50.38915252685547
logit loss: 0.03695851191878319
batch_loss: 19.36560821533203
batch_logit_loss: 0.03511068969964981
batch_box_loss: 19.547697067260742
epoch  1
item loss: 0.020691344514489174
box loss: 0.0
logit loss: 0.020691344514489174
item loss: 49.367286682128906
box loss: 49.340736389160156
logit loss: 0.02654942311346531
item loss: 70.84416961669922
box loss: 70.80704498291016
logit loss: 0.037122540175914764
item loss: 24.23286247253418
box loss: 24.179264068603516
logit loss: 0.053598131984472275
item loss: 0.01958158798

In [21]:
img=Image.open('./val/angular_leafspot14.jpg')
img=transform(img)
img=img.unsqueeze(0)
img=img.to(device)

In [22]:
torch.cuda.empty_cache()

In [23]:
model.mode='detect'
model.eval()
logits,boxes=model(img)


In [24]:
labels=logits.max(dim=-1)[1].squeeze()

target_boxes=boxes.squeeze()[torch.where(labels==1)[0]]
target_logits=logits.squeeze()[torch.where(labels==1)[0]]
target_logits=target_logits[:,1]
display=target_logits.sort(descending=True)[1][0:20]
display_boxes=target_boxes[display]

In [25]:
img_for_drawing=Image.open('./val/angular_leafspot14.jpg')
draw=ImageDraw.Draw(img_for_drawing)
for i in range(len(display_boxes)):
    draw.rectangle([(display_boxes[i][0],display_boxes[i][1]),(display_boxes[i][2],display_boxes[i][3])])
img_for_drawing.show()

In [34]:
image=trainset[3][0].unsqueeze(0)
box=trainset[3][1]
image=image.to(device)
box=box.to(device)
box=[box]
logits,boxes=model(image,box)


In [35]:
labels=logits.max(dim=-1)[1].squeeze()
target_boxes=boxes.squeeze()[torch.where(labels==1)[0]]
target_logits=logits.squeeze()[torch.where(labels==1)[0]]
target_logits=target_logits[:,1]
display=target_logits.sort(descending=True)[1][0:10]
display_boxes=target_boxes[display]


In [36]:
img=image.to('cpu').squeeze().numpy()
img=img.transpose(1,2,0)
img*=255
img=img.astype('uint8')
img=Image.fromarray(img)

In [37]:
draw=ImageDraw.Draw(img)
for i in range(len(display_boxes)):
    # draw.rectangle([(boxes.squeeze()[i][0],boxes.squeeze()[i][1]),(boxes.squeeze()[i][2],boxes.squeeze()[i][3])])
    # draw.rectangle([(target_boxes[i][0],target_boxes[i][1]),(target_boxes[i][2],target_boxes[i][3])])
    draw.rectangle([(display_boxes[i][0],display_boxes[i][1]),(display_boxes[i][2],display_boxes[i][3])])
img.show()

In [17]:
import torch.optim as optim
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01,weight_decay=0.01)

In [18]:
for param in model.backbone.parameters():
    param.requires_grad = False
    
for i, data in enumerate(trainloader):
    image, box = data
    image=image.to(device)
    for i in range(len(box)):
        box[i]=box[i].to(device)
    logit, reg, logit_loss,box_loss = model(image, box)
    total_loss=logit_loss+box_loss
    print('item loss:',total_loss.item())
    total_loss.backward()
    optimizer.step()
    

item loss: 0.6993722915649414
item loss: 0.3465736210346222
item loss: 0.3465736210346222
item loss: 0.3465736210346222
item loss: 0.3465736210346222
item loss: 0.3465736210346222
item loss: 0.3465736210346222


KeyboardInterrupt: 

In [20]:
reg

[tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>),
 tensor([], device='cuda:0', size=(0, 4), grad_fn=<IndexBackward0>)]

In [ ]:
a,b=boxes[0].max(dim=1)

In [ ]:
b

In [ ]:
# 分别计算
iou=IoU(reg[0],box[0],batch_size=1)
#iou[1]=utils.IoU(reg,box[1],batch_size=1)

In [ ]:
iou

In [ ]:
box[0].size()

In [ ]:
target1=torch.where(iou>0.7)[1]# iou>0.7的
target2=set(torch.where(iou[0]<0.3)[0])&set(torch.where(iou[1]<0.3)[0])

In [ ]:
target2

In [ ]:
target=torch.where((iou[1]<0.3)|(iou[1]>0.7))[0]

In [ ]:
target.size()

In [ ]:
iou=iou[:,target]
reg=reg[:,target,:]
cls=cls[:,target,:]

In [ ]:
positive=torch.where(iou>0.7)[1]
negtive=torch.where(iou<0.3)[1]

In [ ]:
box_label=torch.zeros(cls.size())

In [ ]:
box_label[0,positive,1]=1
box_label[0,negtive,0]=1

In [ ]:
a=torch.tensor([1,2,3])
b=torch.tensor([2,3,4])

In [ ]:
a=torch.tensor([[0.2,0.3,1],[1,0.2,0]])

In [ ]:
target=torch.where(a>=1)

In [ ]:
target

In [ ]:
b=torch.rand(3)

In [ ]:
b[target[1]]

In [ ]:
torch.unique(target[1])

In [ ]:
a>=0.2

In [ ]:
a<=0.3

In [ ]:
dir(torch)

In [ ]:
a=()

In [ ]:
b=1
a+=b